In [1]:
# -*- coding: utf-8 -*-
"""
目的：
    從二值遮罩 Raster（例如 0/1 或 0/255）建立面要素（Polygon），
    並提供幾個常見的後處理選配步驟（刪除過小面積、剔除最大面、填洞）。

注意：
    - ArcGIS 的 RasterToPolygon 會依據像元值分割區塊，通常把「目標」設為值＝1（或 255）。
    - 若你的遮罩是 8-bit，請確認背景與前景的像元值（VALUE 欄位）方便後續篩選。
"""

import os
import arcpy

# ========= 使用者需確認/可調參數 =========
# 輸入遮罩 Raster（GeoTIFF 範例）
in_mask_raster = "D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\OL_P1_2312_ClassifyPixel"

# 輸出：Polygon 要素存放的 GDB 與圖層名稱
out_gdb       = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb"
out_fc_name   = "OL_P1_2312_SAML"

# RasterToPolygon 的參數
#   simplify: 是否平滑邊界（"SIMPLIFY" / "NO_SIMPLIFY"）
#   raster_value_field: 轉面後是否保留像元值欄位（通常保留以利後續篩選）
simplify             = "NO_SIMPLIFY"
raster_value_field   = "VALUE"       # 預設為 VALUE；若不需保留可設為 "NO_SIMPLIFY" 的參數不影響此欄
target_value_to_keep = 1             # 只保留值＝1 的區塊（依你的遮罩前景值而調整，如 255）

# （選配）後處理參數
enable_delete_small  = True
small_area_threshold = 0.00001       # 刪除過小面積門檻（使用目前圖層座標單位的面積）
enable_delete_largest = True        # 是否剔除最大面（某些遮罩背景誤差會產生超大面，可剔除）
enable_eliminate_holes = True
hole_area_threshold  = 0.0005        # 填洞門檻（小於此面積的孔洞將被去除）
# =====================================

arcpy.env.overwriteOutput = True

# 確保輸出 GDB 存在
if not arcpy.Exists(out_gdb):
    arcpy.management.CreateFileGDB(os.path.dirname(out_gdb),
                                   os.path.splitext(os.path.basename(out_gdb))[0])

# === (A) Raster → Polygon：核心步驟 ===
def raster_to_polygon(in_ras, out_gdb, out_name,
                      simplify_opt="NO_SIMPLIFY",
                      value_field="VALUE"):
    """
    使用 ArcGIS 的 RasterToPolygon 將遮罩 raster 轉為 polygon。
    重要觀念：
        - 依像元值（VALUE）將連通區塊轉為面。
        - 通常我們僅保留「前景值」的面（例如 1 或 255）。
    備註：
        - conversion.RasterToPolygon 的語法為：
          RasterToPolygon(in_raster, out_polygon_features, simplify, raster_field, create_multipart)
        - 這裡保留 raster_field（VALUE），方便後續只選前景。
    """
    out_fc = os.path.join(out_gdb, out_name)
    arcpy.conversion.RasterToPolygon(
        in_raster=in_ras,
        out_polygon_features=out_fc,
        simplify=simplify_opt,         # "SIMPLIFY" 或 "NO_SIMPLIFY"
        raster_field=value_field      # 將像元值寫入屬性表欄位（預設 "VALUE"）
    )
    return out_fc

# === (B) 僅保留指定像元值（例如：1） ===
def keep_only_target_value(in_fc, value_field, target_value):
    """
    將非目標像元值的面刪除，只保留指定值（例如 1）的區塊。
    使用情境：
        - 遮罩中 1 表示物件、0 表示背景 → 我們只留 VALUE=1 的面。
    """
    if not arcpy.ListFields(in_fc, value_field):
        # 若不存在 VALUE 欄位，代表轉換時未保留；此時可跳過或改用其他策略。
        arcpy.AddWarning(f"找不到欄位 {value_field}，跳過指定值篩選。")
        return

    with arcpy.da.UpdateCursor(in_fc, [value_field]) as ucur:
        for (v,) in ucur:
            if v != target_value:
                ucur.deleteRow()

# === (C) 刪除過小面積 ===
def delete_small_polygons(in_fc, area_thresh):
    """
    刪除面積小於門檻的面（通常用於清除雜訊）。
    注意：
        - 面積單位依圖層座標系統而定（投影座標系統：平方公尺；地理座標系統度：平方度，請避免）
        - 建議在投影座標系統下操作，以確保門檻數值有意義
    """
    deleted = 0
    with arcpy.da.UpdateCursor(in_fc, ["SHAPE@AREA"]) as ucur:
        for (a,) in ucur:
            if a is not None and a < area_thresh:
                ucur.deleteRow()
                deleted += 1
    return deleted

# === (D) 刪除最大面（選配）===
def delete_largest_polygon(in_fc):
    """
    有些遮罩把「背景」也轉成面而且比目標更大，若想一刀剔除最大面可用此法。
    """
    oid_field = arcpy.Describe(in_fc).oidFieldName
    max_oid, max_area = None, float("-inf")
    with arcpy.da.SearchCursor(in_fc, [oid_field, "SHAPE@AREA"]) as cur:
        for oid, area in cur:
            if area is not None and area > max_area:
                max_area = area
                max_oid = oid
    if max_oid is None:
        return 0
    deleted = 0
    with arcpy.da.UpdateCursor(in_fc, [oid_field]) as ucur:
        for (oid,) in ucur:
            if oid == max_oid:
                ucur.deleteRow()
                deleted = 1
                break
    return deleted

# === (E) 填洞（EliminatePolygonPart）===
def eliminate_small_holes(in_fc, out_fc, hole_thresh):
    """
    使用 EliminatePolygonPart 移除小於門檻的「洞」（內環）。
    參數：
        - condition="AREA" 搭配 part_area=hole_thresh：以面積判斷孔洞是否要移除
        - part_option="CONTAINED_ONLY"：只處理完全被包含的孔洞
    說明：
        - 這是針對「洞」的處理，不是對整個面大小做篩選。
    """
    arcpy.management.EliminatePolygonPart(
        in_features=in_fc,
        out_feature_class=out_fc,
        condition="AREA",
        part_area=hole_thresh,
        part_option="CONTAINED_ONLY"
    )

# ===== 主流程 =====
def main():
    # 1) Raster → Polygon
    out_fc = raster_to_polygon(
        in_ras=in_mask_raster,
        out_gdb=out_gdb,
        out_name=out_fc_name,
        simplify_opt=simplify,
        value_field=raster_value_field
    )
    print("RasterToPolygon 輸出：", out_fc)

    # 2) 只保留指定像元值（例如 1）→ 避免把背景值也留著
    keep_only_target_value(out_fc, raster_value_field, target_value_to_keep)
    print(f"已保留 {raster_value_field}={target_value_to_keep} 的多邊形；其他值已刪除。")

    # 3)（選配）刪除過小面
    if enable_delete_small:
        n_small = delete_small_polygons(out_fc, small_area_threshold)
        print(f"已刪除過小面積多邊形 {n_small} 筆（門檻：{small_area_threshold}）。")

    # 4)（選配）剔除最大面（若背景誤轉為巨大面，可一刀剔除）
    if enable_delete_largest:
        n_max = delete_largest_polygon(out_fc)
        print(f"已刪除最大面：{n_max} 筆。")

    # 5)（選配）填洞（輸出到新要素類別，以保留原層）
    if enable_eliminate_holes:
        out_noholes = os.path.join(out_gdb, f"{out_fc_name}_noholes")
        eliminate_small_holes(out_fc, out_noholes, hole_area_threshold)
        print(f"已填洞（< {hole_area_threshold}）：{out_noholes}")

if __name__ == "__main__":
    main()


RasterToPolygon 輸出： D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb\OL_P1_2312_SAML
已保留 VALUE=1 的多邊形；其他值已刪除。
已刪除過小面積多邊形 76239 筆（門檻：1e-05）。
已刪除最大面：1 筆。
已填洞（< 0.0005）：D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb\OL_P1_2312_SAML_noholes
